### RAA_KRX_전종목시세_KOSPI

    1. Web : http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201

    2. page num : [12001] 전종목시세

    3. requests module 사용

    4. ftn : 
        a.False : 해당날짜에 Data 없음
        b.True : pd.DataFrame return
        
    5. 용량문제 : 2017년도부터 적재

In [ ]:
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

import time
import requests
import os
from io import BytesIO

In [ ]:
def ftn(std_date):
    generate_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
    query = {
        "mktId": "STK",
        "trdDd": str(std_date),
        "share": "1",
        "money": "1",
        "csvxls_isNo": "false",
        "name": "fileDown",
        "url": "dbms/MDC/STAT/standard/MDCSTAT01501"
    }

    headers = {
        "Accept": "text/plain, */*; q=0.01",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
        "Content-Length": "120",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Cookie": "__utma=139639017.1078950475.1625955616.1625955616.1625955616.1; __utmc=139639017; __utmz=139639017.1625955616.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmt=1; __utmb=139639017.1.10.1625955616; __smVisitorID=ZBrlOS3pvOH; JSESSIONID=bONcwvNK3NvhGrTAybtyqJinvgYLTBsVTo4hqn8gmIcZE5QIJVMHofUm8wInGO9R.bWRjX2RvbWFpbi9tZGNvd2FwMi1tZGNhcHAxMQ==",
        "Host": "data.krx.co.kr",
        "Origin": "http://data.krx.co.kr",
        "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }

    gen_r = requests.get(generate_url,query,headers = headers)
    time.sleep(0.3)
    down_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"

    form_data = {
        "code" : gen_r.content
    }

    post_r = requests.post(down_url,form_data,headers = headers)
#     time.sleep(0.3)
    post_r.encoding = "CP949"
    tmp_IO = BytesIO(post_r.content)
    tmp = pd.read_csv(tmp_IO,encoding = "CP949",
                      dtype={
                          "종목코드":str
                      })
    tmp["일자"] = str(std_date)
    if tmp.dropna().shape[0]==0:
        return False,tmp
    else:
        return True,tmp

In [ ]:
KOSPI = pd.DataFrame({'종목코드' : [] ,
                    '종목명' : [],
                    '종가' : [],
                    '대비' : [],
                    '등락률':[],
                    '시가' : [],
                    '고가' : [],
                    '저가' : [],
                    '거래량' : [],
                    '거래대금' : [],
                    '시가총액' : [],
                    '상장주식수' : [],
                    '일자' : []})

yyyymmdd = [yy * 10000 + mm *100 + dd for yy in range(1995,2022)for mm in range(1,13) for dd in range(1,32)]
for date in yyyymmdd:
    print(date)
    while 1:
        try :
            A,B = ftn(date)
            break
        except :
            continue
    if A:
        KOSPI = pd.concat([KOSPI,B])

In [ ]:
KOSPI = KOSPI.reset_index(drop = True)
KOSPI = KOSPI.rename(columns = {
            '일자' : '종가일자',
            '대비' : '전일대비증감금액',
            '등락률' : '전일대비주가등락율',
            '거래량' : '거래수량',
            '거래대금' : '거래금액',
            '시가총액' : '시가총금액'
        })

### 기업별 업종명

    1. Web : http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201

    2. page num : [12025] 업종분류 현황

    3. requests module 사용

    4. find_cate :
        a.같은 종목코드인데 업종명이 바뀌는 경우가 있음
        b.해당시점(종가일자)에서 이전 업종명 중 가장 가까운 날짜 선택
        


In [ ]:
def krx(STDATE):
    gen_req_headers = {"Accept": "text/plain, */*; q=0.01",
                         "Accept-Encoding": "gzip, deflate",
                         "Accept-Language": "ko-KR,ko;q=0.9",
                         "Connection": "keep-alive",
                         "Content-Length": "263",
                         "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                         "Cookie": "__utma=139639017.1771199946.1626216000.1626216000.1626216000.1; __utmc=139639017; __utmz=139639017.1626216000.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __smVisitorID=mqaSX0t6UrL; JSESSIONID=NzCZgfqV3jJHxF7ctkxsPFQUbRUzyVwmwczqOaMSqZMWCESSJwIgU1oaGx4huXec.bWRjX2RvbWFpbi9tZGNvd2FwMS1tZGNhcHAxMQ==; finder_listdelisu_finderCd=finder_listdelisu; finder_listdelisu_param1=STK; finder_listdelisu_tbox=011290%2F%EA%B0%91%EC%9D%84%EB%B0%A9%EC%A0%81; finder_listdelisu_codeNm=%EA%B0%91%EC%9D%84%EB%B0%A9%EC%A0%81; finder_listdelisu_codeVal=KR7011290004; finder_comnm_finderCd=finder_comnm; finder_comnm_tbox=005930%2F%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90; finder_comnm_codeNm=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90; finder_comnm_codeVal=00593; finder_comnm_codeVal2=KR7005930003; finder_listdelisu_codeVal2=KR7011290004",
                         "Host": "data.krx.co.kr",
                         "Origin": "http://data.krx.co.kr",
                         "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201",
                         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                         "X-Requested-With": "XMLHttpRequest"}

    gen_form_data = {
                    "mktId": "STK",
                    "trdDd":str(STDATE),
                    "csvxls_isNo": "false",
                    "name": "fileDown",
                    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
    }

    down_req_headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "ko-KR,ko;q=0.9",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Content-Length": "567",
        "Content-Type": "application/x-www-form-urlencoded",
        "Cookie": "__utma=139639017.1771199946.1626216000.1626216000.1626216000.1; __utmc=139639017; __utmz=139639017.1626216000.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __smVisitorID=mqaSX0t6UrL; JSESSIONID=NzCZgfqV3jJHxF7ctkxsPFQUbRUzyVwmwczqOaMSqZMWCESSJwIgU1oaGx4huXec.bWRjX2RvbWFpbi9tZGNvd2FwMS1tZGNhcHAxMQ==; finder_listdelisu_finderCd=finder_listdelisu; finder_listdelisu_param1=STK; finder_listdelisu_tbox=011290%2F%EA%B0%91%EC%9D%84%EB%B0%A9%EC%A0%81; finder_listdelisu_codeNm=%EA%B0%91%EC%9D%84%EB%B0%A9%EC%A0%81; finder_listdelisu_codeVal=KR7011290004; finder_comnm_finderCd=finder_comnm; finder_comnm_tbox=005930%2F%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90; finder_comnm_codeNm=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90; finder_comnm_codeVal=00593; finder_comnm_codeVal2=KR7005930003; finder_listdelisu_codeVal2=KR7011290004",
        "Host": "data.krx.co.kr",
        "Origin": "http://data.krx.co.kr",
        "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    r = requests.post("http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd",gen_form_data,gen_req_headers)
    r = requests.post("http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd",
                  {"code" : r.text},
                  down_req_headers)
    result = pd.read_csv(BytesIO(r.content),encoding="CP949",
                dtype={"종목코드":str})
    result["일자"] = str(STDATE)
    return result

In [ ]:
df = pd.DataFrame({
    "종목코드":[],
    "종목명":[],
    "시장구분":[],
    "업종명":[],
    "종가":[],
    "대비":[],
    "등락률":[],
    "시가총액":[],
    "일자" : []
})

yyyymmdd = [yy * 10000 + mm *100 + dd for yy in range(2001,2022) for mm in range(1,13) for dd in range(1,32)]
for dd in yyyymmdd:
    while 1:
        try :
            df = pd.concat([df,krx(dd)],
                                ignore_index= True,
                                axis = 0)
            print(dd)
            break
        except:
            continue
    if dd >= 20210728:
        break
        
tmp = df.loc[:,["종목코드","업종명","일자"]]

In [ ]:
df_cate = tmp.groupby(["종목코드","업종명"],as_index= False).agg(min)
df_cate["tmp"] = [(df_cate.iloc[x,1],df_cate.iloc[x,2]) for x in range(df_cate.shape[0])]
df_cate = df_cate.sort_values(["종목코드","일자"])
df_cate

In [ ]:
from collections import defaultdict
dic = defaultdict(list)
for num in range(df_cate.shape[0]):
    code,t = df_cate.iloc[num,[0,3]]
    dic[code].append(t)

In [ ]:
def find_cate(code,date):
    tmp = [(cate,dd) for (cate,dd) in dic[code] if int(dd) <= int(date)]
    if len(tmp) > 0:
        return tmp[-1][0]
    
    if len(dic[code]) > 0:
        return dic[code][0][0]
    else :
        return "No searched"

In [ ]:
KOSPI = KOSPI[KOSPI.종가일자.astype(int) >= 20170101]
KOSPI = KOSPI.reset_index()
KOSPI["업종분류명"] = KOSPI.apply(lambda x: find_cate(x["종목코드"],x["종가일자"]),axis=1)
KOSPI = KOSPI.loc[:,["종가일자","종목코드","종목명","업종분류명","시가","고가","저가","종가","전일대비증감금액","전일대비주가등락률","거래수량","거래금액","시가총금액","상장주식수"]]

In [ ]:
KOSPI.to_csv("RAA_KRX전종목시세_KOSP.csv",encoding="utf-8-sig",index = False)